In [2]:
from pathlib import Path
from retinaface import RetinaFace
from tensorflow import keras
from keras_vggface.utils import preprocess_input
import numpy as np

In [24]:
import cv2

In [29]:
from keras_vggface.utils import preprocess_input

In [18]:
cwd = Path(os.path.abspath(''))
sys.path.append(str(cwd.parent.parent))
sys.path.append(str(cwd.parent.parent/"recognition"))

In [19]:
sys.path

['/Users/kmielcar/Documents/projects/faces_classification/classification/vgg',
 '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python38.zip',
 '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8',
 '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/lib-dynload',
 '',
 '/Users/kmielcar/Documents/projects/faces_classification/venv/lib/python3.8/site-packages',
 PosixPath('/Users/kmielcar/Documents/projects/faces_classification/classification'),
 PosixPath('/Users/kmielcar/Documents/projects/faces_classification'),
 '/Users/kmielcar/Documents/projects/faces_classification',
 '/Users/kmielcar/Documents/projects/faces_classification',
 '/Users/kmielcar/Documents/projects/faces_classification',
 '/Users/kmielcar/Documents/projects/faces_classification',
 '/Users/kmielcar/Documents/projects/faces_classification/recognition']

In [20]:
from crop import crop

In [56]:
import importlib

In [60]:
import common

In [61]:
importlib.reload(common)

<module 'common' from '/Users/kmielcar/Documents/projects/faces_classification/common.py'>

In [62]:
from common import EXPECTED_SIZE, GIRLS

In [21]:
test_dir = Path("../../images/test/Grafiti/")
output_dir = Path("../../images/results/")

In [22]:
image_paths = list(test_dir.glob("*jpg"))

In [31]:
model = keras.models.load_model('trained_vgg_on_girls/')

In [66]:
def process_img(img_path:Path):
    detected_faces_desc = RetinaFace.detect_faces(str(img_path))
    faces = detected_faces_desc.values()
    if not faces:
        return

    img_bgr = cv2.imread(str(img_path))
    
    for face_details in faces:
        face_roi = face_details['facial_area']

        cropped_face_img = crop(img_bgr, *face_roi)
        
        cropped_face_img = cropped_face_img.astype('float32')
        sample = np.expand_dims(cropped_face_img, axis=0)
        sample = preprocess_input(sample, version=2)
        
        pred = model.predict(sample)
        max_val_ind = np.argmax(pred[0])
        proba = pred[0][max_val_ind]
        
        text = "{}: {:.2f}%".format(GIRLS[max_val_ind], proba * 100)
        left, top, right, bottom = face_roi
        y = top - 10 if top - 10 > 10 else top + 10
        cv2.rectangle(img_bgr, (left, top), (right, bottom), (0, 255, 0), 2)
        marked_img = cv2.putText(img_bgr, text, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
        
        out_path = output_dir / img_path.name
        cv2.imwrite(str(out_path), marked_img)

In [63]:
test_img.name

'1657797459961.jpg'

In [68]:
for img_path in image_paths:
    process_img(img_path)

1/1 [==============================] - 0s 53ms/step


error: OpenCV(4.6.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [70]:
face_roi

[1525, 1936, 1782, 2288]

In [27]:
img_bgr = cv2.imread(str(test_img))
img_color = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

In [28]:
detected_faces_desc = RetinaFace.detect_faces(str(test_img))

In [44]:
for face_details in detected_faces_desc.values():
    face_roi = face_details['facial_area']
    
    cropped_face_img = crop(img_bgr, *face_roi)
    cropped_face_img = cropped_face_img.astype('float32')
    sample = np.expand_dims(cropped_face_img, axis=0)
    sample = preprocess_input(sample, version=2)
    pred = model.predict(sample)

1/1 [==============================] - 0s 63ms/step


In [45]:
pred

array([[1.7874803e-22, 1.0000000e+00, 1.6545585e-19]], dtype=float32)

In [49]:
ind = np.argmax(pred[0])

In [50]:
proba = pred[0][ind]

In [51]:
text = "{}: {:.2f}%".format(GIRLS[ind], proba * 100)

In [54]:
left, top, right, bottom = face_roi
y = top - 10 if top - 10 > 10 else top + 10
cv2.rectangle(img_bgr, (left, top), (right, bottom), (0, 0, 255), 2)
marked_img = cv2.putText(img_bgr, text, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

In [55]:
cv2.imwrite("out.jpg", marked_img)

True

In [67]:
process_img(test_img)

1/1 [==============================] - 0s 63ms/step


In [69]:
process_img(test_dir/"1657797460255.jpg")

1/1 [==============================] - 0s 51ms/step
